### Data exploration

In [1]:
from scripts.data_exploration import *

# as we have two wrong files: 
# 55_0_0_20170116232725357JPG
# 44_1_4_20170116235150272.pg
file_endings = print_file_endings()

df = get_dataframe()
create_csv(df)
# TODO: add some plots about distribution

Number of JPG files: 24106
Number of DS_STORE files: 1
Number of PG files: 1
Number of 55_0_0_20170116232725357JPG files: 1


### Image preparation

In [ ]:
from scripts.image_helper_functions import *

# THIS IS CURRENTLY ONLY FOR TESTING - WILL BE CLEANED UP

test_image = get_image_from_path('scripts/test.jpg')
grey_image = get_gray_image(test_image)
resized_grey_image = resize_by_height(grey_image, 400)
display_image(resized_grey_image, 'grey and resized image')

# ------------ Model for Age detection ------------ #


# Model requirements for image
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)',
           '(25-32)', '(38-43)', '(48-53)', '(60-100)']
# model_mean = (78.4263377603, 87.7689143744, 114.895847746)

framed_image = get_framed_cropped_face_image(resized_grey_image)
display_image(framed_image, 'framed and cropped image')
# ----- Image preprocessing --------#

# -------Age Prediction---------#
